In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os

mnist = keras.datasets.mnist

(train_images,train_labels),(test_images,test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

# input_mat 設定
train_size = train_images.shape[0]
test_size = test_images.shape[0]
input_mat = np.reshape(train_images, (train_size, train_images.shape[1]*train_images.shape[2]))
# test_images = np.reshape(test_images, (10000, 784))
input_mat.shape

(60000, 784)

In [12]:
# ymat 設定

# 決定 y 的維度
ydim = len(format(max(train_labels), 'b'))
n_train = len(train_labels)

# 初始化 ymat
ymat = np.repeat(0, ydim*n_train).reshape(n_train, ydim)

for i in range(n_train):
    y = np.repeat('0', ydim)
    train_lab = list(format(train_labels[i], 'b'))
    y[ydim - len(train_lab):] = train_lab
    ymat[i, ] = [int(i) for i in y]
ymat.shape

(60000, 4)

In [13]:
import numpy as np
import random

def sigmoid(x):
    return 1/(1 + np.exp(-x))

random.seed(0)

# 參數設定
learning_rate = 0.2

#input_mat = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) #
#ymat = np.array([[0, 0, 1], [1, 1, 0], [1, 1, 1], [0, 0, 1]]) #

n_in = input_mat.shape[1]
n_hidden = [50, 50] # 隱藏層兩層 各有?, ?個neuron
n_out = ymat.shape[1]

# 第1層權重初始值
w = np.random.randn(n_hidden[0], n_in)

# 第2層權重初始值
w2 = np.random.randn(n_hidden[1], n_hidden[0])

# 第3層權重初始值
w3 = np.random.randn(n_out, n_hidden[1])

t = 0
output_, y = ymat[0], ymat[0]+1

while min(output_ == y) != True:
    
    input_ = input_mat[t % input_mat.shape[0]]
    y = ymat[t % input_mat.shape[0]]

    ### 正向傳播

    # 計算output
    hidden_1 = sigmoid(np.matmul(w, input_))
    hidden_2 = sigmoid(np.matmul(w2, hidden_1))
    output_ = sigmoid(np.matmul(w3, hidden_2))



    ### 誤差反向傳播

    # 第3層權重delta計算
    delta3 = (y - output_) * output_ * (1 - output_)

    # 第2層權重delta計算
    delta2 = hidden_2 * (1 - hidden_2) * np.matmul(delta3, w3)

    # 第3層權重delta計算
    delta1 = hidden_1 * (1 - hidden_1) * np.matmul(delta2, w2)

    # 第3層權重修正
    w3 += learning_rate * np.tensordot(delta3, hidden_2, axes = 0)

    # 第2層權重修正
    w2 += learning_rate * np.tensordot(delta2, hidden_1, axes = 0)

    # 第1層權重修正
    w += learning_rate * np.tensordot(delta1, input_, axes = 0)

    t += 1

    if t > 500000 or (abs(output_ - y) < 0.000001 ).all():
        break


In [14]:
test_mat = np.reshape(test_images, (10000, 784)).T
test_hidden_1 = sigmoid(np.matmul(w, test_mat))
test_hidden_2 = sigmoid(np.matmul(w2, test_hidden_1))
test_output_ = sigmoid(np.matmul(w3, test_hidden_2))

In [15]:
output_raw = np.round_(test_output_.T)
output_y = np.zeros(output_raw.shape[0])
for i in range(output_raw.shape[0]):
    for j in range(ydim):
        output_y[i] += output_raw[i, ydim-j-1] * (2**j)


In [16]:
sum(test_labels == output_y)/output_raw.shape[0]

0.9337

In [ ]:
# 結果寫入檔案